# Example Stowage Planning with RL

Imports for usage

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
module_path = str(os.getcwd())+'\\out\\'

from env import roroDeck
from agent import sarsa, tdq
from analysis import *
from algorithms.Algorithms import *
from valuation import evaluator as evm
#from valuation import evaluation
#from viz import *
from agent.dqn import DQNAgent
import pickle



from keras.models import Sequential, load_model
import numpy as np

Using TensorFlow backend.


## Load a model and show input data

In [3]:
env = roroDeck.RoRoDeck()
agent = tdq.TDQLearning(env,module_path)

agent.load_model(module_path+"20200911\\1307\\1307TDQ_L8_R12_Rf1_A5.p")

env = agent.env
evaluator = evm.Evaluator(env.vehicle_data, env.grid)


for info in agent.q_table["ModelParam"]:
    print(info +': '+ str(agent.q_table["ModelParam"][info]))


Algorithm: Time Difference Q-Learning
GAMMA: 0.999
ALPHA: 0.1
Episodes: 1000000
EnvLanes:: 8
EnvRows: 12
VehicleData: [[ 0  1  2  3  4]
 [ 5  5 -1 -1  2]
 [ 1  1  0  0  1]
 [ 1  2  1  2  2]
 [ 2  3  2  3  2]
 [ 0  0  0  0  1]]
TrainingTime: 7335.581272125244


In [3]:
#env = roroDeck.RoRoDeck(False,12,14)
#evaluator = evm.Evaluator(env.vehicle_data, env.grid)
#agent = DQNAgent(env,module_path)

#agent.load_model(module_path+"20200618\\1644\\1644DQN_L12_R14_Rf1_A5")
#agent.q_eval.summary()

In [4]:
#env = roroDeck.RoRoDeck(False,8,10)
#evaluator = evm.Evaluator(env.vehicle_data, env.grid)
#DQN_agent = DQNAgent(gamma=0.999, epsilon=1.0, alpha=0.0004, input_dims=np.shape(env.reset())[0], n_actions=5, mem_size=10000000, batch_size=32, epsilon_end=0.01, epsilon_dec= 0.999992)
#DQN_agent.load_model(module_path+"20200428\\1045\\104520200427stochasticdqn_model.h5")


## Show the best stowage plan the agent found

In [4]:
env.reset()
#print(np.shape(env.current_state))
#DQN_agent.execute(env)
agent.execute(env)
evaluation = evaluator.evaluate(env.get_stowage_plan())
env.render()
print(evaluation)

-----------Loading Sequence----------------------------------------------------------------
X	X	X	X	X	X	X	X	

X	X	X	1	2	X	X	X	

X	X	3	1	2	4	X	X	

X	5	3	1	2	4	6	X	

7	5	3	8	9	10	6	11	

7	5	12	8	9	10	6	11	

13	14	12	15	16	17	18	19	

13	14	12	15	16	17	18	19	

20	14	21	22	23	24	18	25	

20	26	21	22	23	24	27	25	

28	26	29	30	31	32	27	33	

28	26	29	30	31	32	27	33	

-----------VehicleType--------------------------------------------------------------------
X	X	X	X	X	X	X	X	

X	X	X	1	3	X	X	X	

X	X	1	1	3	0	X	X	

X	1	1	1	3	0	1	X	

4	1	1	0	0	0	1	0	

4	1	1	0	0	0	1	0	

4	3	1	2	2	2	3	2	

4	3	1	2	2	2	3	2	

2	3	2	2	2	2	3	2	

2	3	2	2	2	2	3	2	

2	3	2	2	2	2	3	2	

2	3	2	2	2	2	3	2	

-----------Destination--------------------------------------------------------------------
X	X	X	X	X	X	X	X	

X	X	X	2	2	X	X	X	

X	X	2	2	2	1	X	X	

X	2	2	2	2	1	2	X	

2	2	2	1	1	1	2	1	

2	2	2	1	1	1	2	1	

2	2	2	1	1	1	2	1	

2	2	2	1	1	1	2	1	

1	2	1	1	1	1	2	1	

1	2	1	1	1	1	2	1	

1	2	1	1	1	1	2	1	

1	2	1	1	1	1	2	1	


Mandatory Cargo Loaded: 

Show the loading sequence which constructed this stowage plan 

In [5]:
print(env.loading_sequence)

Loading Sequence of RORO-Deck (Lanes: 8 Rows: 12)

1. Load Vehicle Type 	 1 	 in Lane: 	 3 	 Row: 	 1 
2. Load Vehicle Type 	 3 	 in Lane: 	 4 	 Row: 	 1 
3. Load Vehicle Type 	 1 	 in Lane: 	 2 	 Row: 	 2 
4. Load Vehicle Type 	 0 	 in Lane: 	 5 	 Row: 	 2 
5. Load Vehicle Type 	 1 	 in Lane: 	 1 	 Row: 	 3 
6. Load Vehicle Type 	 1 	 in Lane: 	 6 	 Row: 	 3 
7. Load Vehicle Type 	 4 	 in Lane: 	 0 	 Row: 	 4 
8. Load Vehicle Type 	 0 	 in Lane: 	 3 	 Row: 	 4 
9. Load Vehicle Type 	 0 	 in Lane: 	 4 	 Row: 	 4 
10. Load Vehicle Type 	 0 	 in Lane: 	 5 	 Row: 	 4 
11. Load Vehicle Type 	 0 	 in Lane: 	 7 	 Row: 	 4 
12. Load Vehicle Type 	 1 	 in Lane: 	 2 	 Row: 	 5 
13. Load Vehicle Type 	 4 	 in Lane: 	 0 	 Row: 	 6 
14. Load Vehicle Type 	 3 	 in Lane: 	 1 	 Row: 	 6 
15. Load Vehicle Type 	 2 	 in Lane: 	 3 	 Row: 	 6 
16. Load Vehicle Type 	 2 	 in Lane: 	 4 	 Row: 	 6 
17. Load Vehicle Type 	 2 	 in Lane: 	 5 	 Row: 	 6 
18. Load Vehicle Type 	 3 	 in Lane: 	 6 	 Row: 	 6 
19. 

## Construct a stowage plan sequentially

Example of a an human-agent interaction:
Agent shows the predictions for each action in a given state and recommends the best one.
If the Environment is set to deterministic behaviour the stowage plan above is reconstructed if all recommendations are obeyed

Usage (Type the following on the Keyboard and press Enter):

0,1,2 &nbsp; &nbsp; &nbsp; &nbsp; *number from the list of possible actions*

r &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; *show the current state of the RORO-deck*

b &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;*follow the best action*

f &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;*stop interaction by only following the recommendations of agent*

q &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;*quit the execution*




In [10]:
state = env.reset()
done = False
counter = 1
while not done:
    print("\n"+str(counter)+". Vehicle\nPossible Actions for Lane "+str(env.current_Lane)
          +"\n"+str(env.possible_actions))
    source = []
    for action in env.possible_actions:
        source += [round(float(agent.predict(state,action)),2)]
    best_action = agent.max_action(state,env.possible_actions)
    
    #print(env.possible_actions)
    #source = DQN_agent.q_eval.predict(state[np.newaxis, :])
    #best_action = DQN_agent.maxAction(state,env.possible_actions)
    #print("Prediction of Agent:\n"+str(source[0][env.possible_actions]))
    print("Prediction of Agent:\n"+str(source))
    print("--> Choose: "+str(best_action))
    action = input() 
    try:
        action = int(action)
        if int(action) in env.possible_actions:
            state, reward, done, info = env.step(int(action))
            counter += 1            
    except:
        if action == 'b':
            state, reward, done, info = env.step(int(best_action))
            counter += 1
        elif action == 'f':
            #DQN_agent.execute(env)
            agent.execute(env)
            #agent.execute()
            break
        elif action == 'q':
            print("Quit execution mode")
            break
        elif action == 'r':
            env.render()
env.render()
print("\n\n")
evaluation = evaluator.evaluate(env.get_stowage_plan())
print(evaluation)
print("\n\n")
print(env.loading_sequence)


1. Vehicle
Possible Actions for Lane 3
[0 1 2 3]
Prediction of Agent:
[23.17, 23.49, 23.17, 23.26]
--> Choose: 1
f
-----------Loading Sequence----------------------------------------------------------------
X	X	X	X	X	X	X	X	

X	X	X	1	2	X	X	X	

X	X	3	1	2	4	X	X	

X	5	3	1	2	4	6	X	

7	5	3	8	9	10	6	11	

7	5	12	8	9	10	6	11	

13	14	12	15	16	17	18	19	

13	14	12	15	16	17	18	19	

20	14	21	22	23	24	18	25	

20	26	21	22	23	24	27	25	

28	26	29	30	31	32	27	33	

28	26	29	30	31	32	27	33	

-----------VehicleType--------------------------------------------------------------------
X	X	X	X	X	X	X	X	

X	X	X	1	3	X	X	X	

X	X	1	1	3	0	X	X	

X	1	1	1	3	0	1	X	

4	1	1	0	0	0	1	0	

4	1	1	0	0	0	1	0	

4	3	1	2	2	2	3	2	

4	3	1	2	2	2	3	2	

2	3	2	2	2	2	3	2	

2	3	2	2	2	2	3	2	

2	3	2	2	2	2	3	2	

2	3	2	2	2	2	3	2	

-----------Destination--------------------------------------------------------------------
X	X	X	X	X	X	X	X	

X	X	X	2	2	X	X	X	

X	X	2	2	2	1	X	X	

X	2	2	2	2	1	2	X	

2	2	2	1	1	1	2	1	

2	2	2	1	1	1	2	1	

2	2	2	1	1	1	2	1	
